# Introduction

After much consideration, we decided to use the ``gurobipy`` package to solve this LP problem

In [19]:
from gurobipy import *
import pandas

i_set = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
j_set = ['1s1', '2s1','3m1','4m1','5t1','6t1','5t2','6t2','5t3','6t3','7w1','8w1','7w2','8w2', '5b', '6b', '7b', '8b']

# create a LP model
model = Model()

# Variables

Now we can set the variables

In [20]:
# Set Variables
x = {}
for i in i_set:
    for j in j_set:
        x[str(i)+'~'+j] = model.addVar(vtype=GRB.BINARY, name=str(i) + '~' + j)
        
model.update()
x

{'10~1s1': <gurobi.Var 10~1s1>,
 '10~2s1': <gurobi.Var 10~2s1>,
 '10~3m1': <gurobi.Var 10~3m1>,
 '10~4m1': <gurobi.Var 10~4m1>,
 '10~5b': <gurobi.Var 10~5b>,
 '10~5t1': <gurobi.Var 10~5t1>,
 '10~5t2': <gurobi.Var 10~5t2>,
 '10~5t3': <gurobi.Var 10~5t3>,
 '10~6b': <gurobi.Var 10~6b>,
 '10~6t1': <gurobi.Var 10~6t1>,
 '10~6t2': <gurobi.Var 10~6t2>,
 '10~6t3': <gurobi.Var 10~6t3>,
 '10~7b': <gurobi.Var 10~7b>,
 '10~7w1': <gurobi.Var 10~7w1>,
 '10~7w2': <gurobi.Var 10~7w2>,
 '10~8b': <gurobi.Var 10~8b>,
 '10~8w1': <gurobi.Var 10~8w1>,
 '10~8w2': <gurobi.Var 10~8w2>,
 '11~1s1': <gurobi.Var 11~1s1>,
 '11~2s1': <gurobi.Var 11~2s1>,
 '11~3m1': <gurobi.Var 11~3m1>,
 '11~4m1': <gurobi.Var 11~4m1>,
 '11~5b': <gurobi.Var 11~5b>,
 '11~5t1': <gurobi.Var 11~5t1>,
 '11~5t2': <gurobi.Var 11~5t2>,
 '11~5t3': <gurobi.Var 11~5t3>,
 '11~6b': <gurobi.Var 11~6b>,
 '11~6t1': <gurobi.Var 11~6t1>,
 '11~6t2': <gurobi.Var 11~6t2>,
 '11~6t3': <gurobi.Var 11~6t3>,
 '11~7b': <gurobi.Var 11~7b>,
 '11~7w1': <gurobi.Var

# Objective function

Now we can write down the objective function

In [15]:
# Set objective
model.setObjective(quicksum(x[key] for key in x.keys()), GRB.MINIMIZE)

# Constraints

Now we can write down the constraints

In [16]:
# Add constraints
c = {}

# 1. meet the conference demands
conference_demands = [2,2,2,2,2,2,1,1,1,1,2,2,1,1]
c_i = 0  # conference index
conference_shift = j_set[:-4] # without '5b'...
for j in conference_shift:
    c['x_i~' + j] = model.addConstr(quicksum(x[str(i)+'~' + j] for i in i_set) >= conference_demands[c_i], name='x_i~' + j + ' >= ' + str(conference_demands[c_i]))
    c_i += 1

# 2. meet the volunteers availability
availability = pandas.read_excel('Availability.xlsx').transpose()
for i in i_set:
    for j in j_set:
        c[str(i)+'~'+j +" <= A_" + str(i) + '~' + j] = model.addConstr(x[str(i)+'~'+j] <= availability.loc[i, j], name=str(i)+'~'+j +" <= A_" + str(i) + '~' + j)


# 3. special constraints
# 3.1 student 2 and student 5
c["x_2~j <= 1"] = model.addConstr(quicksum(x['2~' + j] for j in j_set) <= 1, name="x_2~j <= 1")
c["x_5~j <= 1"] = model.addConstr(quicksum(x['5~' + j] for j in j_set) <= 1, name="x_5~j <= 1")

# 3.2 
c["x_i~5b >= 1"] = model.addConstr(quicksum(x[str(i)+'~5b'] for i in i_set) >= 1, name="x_i~5b >= 1")
c["x_i~6b >= 1"] = model.addConstr(quicksum(x[str(i)+'~6b'] for i in i_set) >= 1, name="x_i~6b >= 1")
c["x_i~7b >= 1"] = model.addConstr(quicksum(x[str(i)+'~7b'] for i in i_set) >= 1, name="x_i~7b >= 1")
c["x_i~8b >= 1"] = model.addConstr(quicksum(x[str(i)+'~8b'] for i in i_set) >= 1, name="x_i~8b >= 1")

# 4. no students can be assigned to different location during the same shift
for i in i_set:
    all_location = ['5t1', '5t2', '5t3', '5b']
    c["x_" + str(i) + "~5(k) <= 1"] = model.addConstr(quicksum(x[str(i)+'~' + k] for k in all_location) <= 1, name="x_" + str(i) + "~5(k) <= 1")
    all_location = ['6t1', '6t2', '6t3', '6b']
    c["x_" + str(i) + "~6(k) <= 1"] = model.addConstr(quicksum(x[str(i)+'~' + k] for k in all_location) <= 1, name="x_" + str(i) + "~6(k) <= 1")
    all_location = ['7w1', '7w2', '7b']
    c["x_" + str(i) + "~7(k) <= 1"] = model.addConstr(quicksum(x[str(i)+'~' + k] for k in all_location) <= 1, name="x_" + str(i) + "~7(k) <= 1")
    all_location = ['8w1', '8w2', '8b']
    c["x_" + str(i) + "~8(k) <= 1"] = model.addConstr(quicksum(x[str(i)+'~' + k] for k in all_location) <= 1, name="x_" + str(i) + "~8(k) <= 1")


model.update()

c

{'10~1s1 <= A_10~1s1': <gurobi.Constr 10~1s1 <= A_10~1s1>,
 '10~2s1 <= A_10~2s1': <gurobi.Constr 10~2s1 <= A_10~2s1>,
 '10~3m1 <= A_10~3m1': <gurobi.Constr 10~3m1 <= A_10~3m1>,
 '10~4m1 <= A_10~4m1': <gurobi.Constr 10~4m1 <= A_10~4m1>,
 '10~5b <= A_10~5b': <gurobi.Constr 10~5b <= A_10~5b>,
 '10~5t1 <= A_10~5t1': <gurobi.Constr 10~5t1 <= A_10~5t1>,
 '10~5t2 <= A_10~5t2': <gurobi.Constr 10~5t2 <= A_10~5t2>,
 '10~5t3 <= A_10~5t3': <gurobi.Constr 10~5t3 <= A_10~5t3>,
 '10~6b <= A_10~6b': <gurobi.Constr 10~6b <= A_10~6b>,
 '10~6t1 <= A_10~6t1': <gurobi.Constr 10~6t1 <= A_10~6t1>,
 '10~6t2 <= A_10~6t2': <gurobi.Constr 10~6t2 <= A_10~6t2>,
 '10~6t3 <= A_10~6t3': <gurobi.Constr 10~6t3 <= A_10~6t3>,
 '10~7b <= A_10~7b': <gurobi.Constr 10~7b <= A_10~7b>,
 '10~7w1 <= A_10~7w1': <gurobi.Constr 10~7w1 <= A_10~7w1>,
 '10~7w2 <= A_10~7w2': <gurobi.Constr 10~7w2 <= A_10~7w2>,
 '10~8b <= A_10~8b': <gurobi.Constr 10~8b <= A_10~8b>,
 '10~8w1 <= A_10~8w1': <gurobi.Constr 10~8w1 <= A_10~8w1>,
 '10~8w2 <= A

# Solution

Now, let's get the results

In [17]:
model.optimize()
print()
print("*****Thus the objective value is " + str(model.ObjVal))

Optimize a model with 328 rows, 252 columns and 736 nonzeros
Variable types: 0 continuous, 252 integer (252 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 26
Presolve removed 280 rows and 121 columns
Presolve time: 0.00s
Presolved: 48 rows, 131 columns, 262 nonzeros
Variable types: 0 continuous, 131 integer (131 binary)

Root relaxation: cutoff, 17 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0        26.00000   26.00000  0.00%     -    0s

Explored 0 nodes (17 simplex iterations) in 0.02 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 26 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.600000000000e+01, best bound 2.600000000000e+01, g

# The resulting variables

In [18]:
x

{'10~1s1': <gurobi.Var 10~1s1 (value 0.0)>,
 '10~2s1': <gurobi.Var 10~2s1 (value 0.0)>,
 '10~3m1': <gurobi.Var 10~3m1 (value 0.0)>,
 '10~4m1': <gurobi.Var 10~4m1 (value 0.0)>,
 '10~5b': <gurobi.Var 10~5b (value 0.0)>,
 '10~5t1': <gurobi.Var 10~5t1 (value 0.0)>,
 '10~5t2': <gurobi.Var 10~5t2 (value 0.0)>,
 '10~5t3': <gurobi.Var 10~5t3 (value 0.0)>,
 '10~6b': <gurobi.Var 10~6b (value 0.0)>,
 '10~6t1': <gurobi.Var 10~6t1 (value 0.0)>,
 '10~6t2': <gurobi.Var 10~6t2 (value 0.0)>,
 '10~6t3': <gurobi.Var 10~6t3 (value 0.0)>,
 '10~7b': <gurobi.Var 10~7b (value 0.0)>,
 '10~7w1': <gurobi.Var 10~7w1 (value 0.0)>,
 '10~7w2': <gurobi.Var 10~7w2 (value 0.0)>,
 '10~8b': <gurobi.Var 10~8b (value 0.0)>,
 '10~8w1': <gurobi.Var 10~8w1 (value 0.0)>,
 '10~8w2': <gurobi.Var 10~8w2 (value 0.0)>,
 '11~1s1': <gurobi.Var 11~1s1 (value 0.0)>,
 '11~2s1': <gurobi.Var 11~2s1 (value 0.0)>,
 '11~3m1': <gurobi.Var 11~3m1 (value 0.0)>,
 '11~4m1': <gurobi.Var 11~4m1 (value 1.0)>,
 '11~5b': <gurobi.Var 11~5b (value 0.0)>